# ガウス混合モデルを使ったMCMC

In [0]:
import numpy as np
import tensorflow as tf
from mpl_toolkits.mplot3d import Axes3D
#from tensorflow_probability import distributions as tfd
from tensorflow_probability import positive_semidefinite_kernels as tfk
import matplotlib.pyplot as plt
import tensorflow_probability as tfp
from scipy.stats import wishart, chi2
tfd = tfp.distributions

def reset_session():
  """Creates a new global, interactive session in Graph-mode."""
  global sess
  try:
    tf.reset_default_graph()
    sess.close()
  except:
    pass
  sess = tf.InteractiveSession()

reset_session()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



## ギブスサンプリングでパラメータと潜在変数を推定する

In [0]:
from sympy import *
from sympy.stats import *
import numpy as np
from numpy.linalg import inv
from numpy.random import *
from matplotlib import pyplot as plt
import math
np.random.seed(0)

'''
ガウス混合モデルをギブスサンプリングで推定する
'''

#-#-# ハイパーパラメータ #-#-#
N = 200 # サンプル数
D = 2 # 次元数
K = 2 # クラスタ数


#-#-# データXを生成する #-#-#
mean_1 = np.array([6., 6.])
mean_2 = np.array([11., 11.])

abc = 0.5
qwe = 2
covar_1 = 1/(1-abc**2) * np.array([[1, abc],[abc, 1]])
covar_2 = 1/(1-qwe**2) * np.array([[1, qwe],[qwe, 1]])

X_1 = multivariate_normal(mean_1, covar_1, size=N)
X_2 = multivariate_normal(mean_2, covar_2, size=N)
X = np.vstack([X_1, X_2])
#plt.plot(X[:,0], X[:,1], 'o')
#plt.show()


#-#-# ギブスサンプリングする #-#-#
# まずは必要なパラメータを初期化する
mean = np.zeros((D, K))
covar00 = 1/(1-0.8**2) * np.array([[1, 0.8],[0.8, 1]])
covar = np.dstack((covar00, covar00))
pi = np.zeros((K, 1)) + (1 / K)

tmp = np.zeros((N*2, K))

# NとKで場合分けしないといけないね
for n in range(N*2):
	for k in range(K):
		precision = np.matrix(covar[:,:,k]).I
		part = np.dot(np.dot(X[n] - mean[:,k], precision), (X[n] - mean[:,k]).T)
		(sign, logdet) = np.linalg.slogdet(precision)
		tmp[n][k] = -0.5*part + 0.5*logdet + np.log(pi[k,:])
logZ = - np.log(np.sum(np.exp(tmp),axis=1)).reshape(-1, 1)
eta = np.exp(tmp + logZ)

sampled_lmd = []
sampled_pi = []

# Sをサンプリングする
S = np.zeros((N*2, K))
for n in range(N*2):
	S[n] = np.random.multinomial(n=1, pvals=eta[n], size=1)
	
# 精度行列をサンプリングする
for k in range(K):
	
	m = np.ones((1,D))
	beta = 1
	W = np.matrix(np.array([[1,2],	 
											    [3,4]]))
	nu = D
	
	beta_ = np.sum(S[:,k], axis=0) + beta
	
	SXX = np.zeros((N*2, K))
	SX = np.zeros((N*2, K))
	
	for n in range(N*2):
		SX[n] = S[n][k] * X[n].reshape(1,-1)
		SXX[n] = S[n][k] * np.dot(X[n].reshape(1,-1), X[n].reshape(1,-1).T)
	
	m_ = (1/beta_) * (np.sum(SX, axis=0)[k] + beta*np.zeros((1,D)))
	W_ = np.sum(SXX, axis=0)[k] + beta*np.dot(m,m.T) - beta_*np.dot(m_,m_.T) + W.I
	nu_ = np.sum(SXX, axis=0)[k] + D
	print(nu_)
	print(W_)

# 平均パラメータをサンプリングする
#mu = multivariate_normal(m_, , size=N)







29886.980102926565
[[2768.58152827 2771.58152827]
 [2772.08152827 2770.08152827]]
33386.15830987383
[[2928.29081631 2931.29081631]
 [2931.79081631 2929.79081631]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: covariance is not symmetric positive-semidefinite.


In [0]:
A = sess.run(tfd.Wishart(df=nu_, scale_tril=W_.I).sample(1))

cov = np.matrix(beta_ * A).I
mmm = multivariate_normal(m_[0], cov, size=(1))






[[8.53223382 8.5343145 ]]
[8.53586123 8.53586123]
